## DATA PREPROCESSING

In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,confusion_matrix

In [17]:
df = pd.read_excel("category.xlsx",sheet_name= [1,4])
loc_to_cat = df[1]
loc_to_cat['category1'] = loc_to_cat['category1'].str.lower()
allUserData = df[4]
#userData = [df[3], df[4], df[5], df[6], df[7]]

#for i in userData[1:]:
#    allUserData = allUserData.append(i)
allUserData.head()

,user_id,weekday,date,start_time,end_date,end_time,duration,category,category_id,location_name,location_category,my_place,location_id,place_id
0,2,2,2020-06-29,16:00:00,2020-06-29,19:00:00,03:00:00,meeting,13,路易莎,cafe,NaN,4.0,NaN
1,2,2,2020-06-29,19:00:00,2020-06-29,20:00:00,01:00:00,food,1,全哥鵝肉小炒,restaurant,NaN,5.0,NaN
2,2,2,2020-06-29,20:00:00,2020-06-29,20:30:00,00:30:00,leisure,10,NaN,NaN,home,NaN,1.0
3,2,2,2020-06-29,20:30:00,2020-06-29,21:30:00,01:00:00,shopping,12,Miramar,Shopping mall,NaN,6.0,NaN
4,2,2,2020-06-29,21:30:00,2020-06-29,23:30:00,02:00:00,study,5,NaN,NaN,home,NaN,1.0


In [18]:
allUserData['location']= allUserData['location_id'].combine_first(allUserData['place_id'])
#drop commute
indexNames = allUserData[(allUserData['category_id'] == 17) ].index
allUserData.drop(indexNames,inplace=True)

allUserData['location'].fillna("99", inplace = True)
allUserData = allUserData.drop(['date','end_date','end_time','location_name','location_category','my_place','location_id','place_id'], 1)
allUserData = allUserData.reset_index(drop = True)

In [19]:
def get_hours(ts):
    a = ts.hour
    return a

def get_mins(ts):
    a = ts.minute
    return a

def get_duration(ts):
    a = ts.hour*60 + ts.minute
    return a 

#allUserData['start_mins'] = allUserData['start_time'].apply(get_mins)
allUserData['start_time'] = allUserData['start_time'].apply(get_duration)
allUserData['duration'] = allUserData['duration'].apply(get_duration)
allUserData = allUserData[['user_id','weekday','start_time','duration','location','category','category_id']]
allUserData.tail()

,user_id,weekday,start_time,duration,location,category,category_id
444,2,5,50,495,1,sleep,3
445,2,5,545,15,1,leisure,10
446,2,5,570,150,2,work,6
447,2,5,780,360,3,meeting,13
448,2,5,1200,240,1,leisure,10


## Random Forest Model

In [28]:
train_df = allUserData.loc[0:299]
test_df = allUserData.loc[300:]
print("Number of traing data is :",len(train_df))
print("Number of testing data is :",len(test_df))

Number of traing data is : 300
Number of testing data is : 149


In [29]:
# without diff
X_train = train_df.iloc[:, np.r_[0:4]].values
y_train = train_df.iloc[:, 6].values

X_test = test_df.iloc[:, np.r_[0:4]].values
y_test = test_df.iloc[:, 6].values

In [30]:
clf = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=50, verbose=0,
            warm_start=False)
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=50, verbose=0, warm_start=False)

In [31]:
clf.score(X_test, y_test)

0.6308724832214765

In [32]:
imp = clf.feature_importances_
imp

array([0.        , 0.11707604, 0.48461265, 0.3983113 ])

In [26]:
predictions = clf.predict(X_test)
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           1       0.36      0.71      0.48        21
           3       1.00      0.95      0.97        19
           4       0.00      0.00      0.00         0
           6       0.50      0.82      0.62        11
           9       0.80      0.80      0.80         5
          10       0.80      0.71      0.75        58
          11       0.50      0.12      0.20         8
          12       0.44      0.25      0.32        16
          13       1.00      0.33      0.50         6
          16       0.00      0.00      0.00         2
          18       0.00      0.00      0.00         3

   micro avg       0.63      0.63      0.63       149
   macro avg       0.49      0.43      0.42       149
weighted avg       0.67      0.63      0.62       149



/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [27]:
print(confusion_matrix(y_test,predictions))

[[15  0  0  0  0  4  1  1  0  0  0]
 [ 0 18  0  0  0  1  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  9  0  1  0  0  0  1  0]
 [ 0  0  0  0  4  1  0  0  0  0  0]
 [ 8  0  1  5  0 41  0  3  0  0  0]
 [ 5  0  0  0  0  2  1  0  0  0  0]
 [10  0  0  0  1  1  0  4  0  0  0]
 [ 0  0  0  4  0  0  0  0  2  0  0]
 [ 2  0  0  0  0  0  0  0  0  0  0]
 [ 2  0  0  0  0  0  0  1  0  0  0]]


In [33]:
predictions

array([12,  1,  1, 10, 10, 10,  3, 10, 10,  1, 10, 11,  1,  1,  4, 10,  3,
       10,  6, 10, 10, 10, 10,  1,  6,  1, 12, 10, 10,  3,  6,  6, 10,  9,
       10,  3,  6,  1,  1, 13, 10, 10,  9, 10,  3,  6,  6,  1, 10, 10,  3,
       10,  1,  1, 12,  1,  1, 11,  1, 10,  3, 10,  3, 10, 12, 12,  1,  6,
        1,  1, 12,  1, 10,  3, 10,  6,  1, 10, 12,  9, 10,  3,  6,  6,  1,
       10,  9, 10, 10,  3, 10,  6,  1, 13,  1,  1,  1, 10, 10,  3, 10,  1,
       16,  1, 10,  3, 10,  1,  1, 10,  1,  1,  1, 10,  3, 10,  1, 10, 12,
       10,  3, 10,  6,  1, 12, 10,  3, 10,  1,  6,  1, 10,  3,  6,  6,  1,
        1,  1,  1,  1,  1, 10,  9, 10,  3, 10,  6,  6, 10])

## User Data

In [110]:
location = pd.read_csv("0908.csv")
location.tail()

,location_id,longitude,latitude,start_date,start_time,weekday,duration,speed,name1,name2,name3,name4,name5,category1,category2,category3,category4,category5
1119,1120,121.522094,25.062802,2020-09-08,11:26,3,0.0,-1.0,LOUISA COFFEE 路易莎咖啡 (民權西門市),Artco De Cafe,最佳風情國際婚紗,Jessie Wa Brunch. Coffee,Judy Judy cultural and creative wedding photog...,cafe,cafe,point_of_interest,restaurant,point_of_interest
1120,1121,121.522291,25.062872,2020-09-08,11:52,3,0.0,-1.0,LOUISA COFFEE 路易莎咖啡 (民權西門市),Jessie Wa Brunch. Coffee,Artco De Cafe,Judy Judy cultural and creative wedding photog...,威成文具批發,cafe,restaurant,cafe,point_of_interest,point_of_interest
1121,1122,121.522291,25.062872,2020-09-08,11:52,3,0.0,-1.0,LOUISA COFFEE 路易莎咖啡 (民權西門市),Jessie Wa Brunch. Coffee,Artco De Cafe,Judy Judy cultural and creative wedding photog...,威成文具批發,cafe,restaurant,cafe,point_of_interest,point_of_interest
1122,1123,121.522291,25.062872,2020-09-08,11:52,3,0.0,-1.0,LOUISA COFFEE 路易莎咖啡 (民權西門市),Jessie Wa Brunch. Coffee,Artco De Cafe,Judy Judy cultural and creative wedding photog...,威成文具批發,cafe,restaurant,cafe,point_of_interest,point_of_interest
1123,1124,121.522291,25.062872,2020-09-08,11:52,3,0.0,-1.0,LOUISA COFFEE 路易莎咖啡 (民權西門市),Jessie Wa Brunch. Coffee,Artco De Cafe,Judy Judy cultural and creative wedding photog...,威成文具批發,cafe,restaurant,cafe,point_of_interest,point_of_interest


In [116]:
stopLoc = location.drop(['category3','category4','category5'], 1)

def string_to_mins(ts):
    a = int(ts.split(':')[0])
    b = int(ts.split(':')[1])
    c = a*60 + b
    return c

#stopLoc['start_time'] = pd.to_datetime(stopLoc['start_time'], format='%H:%M', errors='ignore').apply(string_to_mins)
stopLoc['longitude'] = round(stopLoc['longitude'],3)
stopLoc['latitude'] = round(stopLoc['latitude'],3)
stopLoc

,location_id,longitude,latitude,start_date,start_time,weekday,duration,speed,name1,name2,name3,name4,name5,category1,category2
0,1,121.576,24.986,2020/07/22,11:00,4,NaN,NaN,myhome,myhome1,myhome2,myhome3,myhome4,c1,c2
1,2,121.443,24.433,2020/07/23,12:00,5,NaN,NaN,apple,apple1,apple2,apple3,apple4,c1,c2
2,3,121.123,24.765,2020/07/24,13:00\n,6,NaN,NaN,banana,banana1,banana2,banana3,banana4,c1,c2
3,4,121.563,25.087,2020-09-04,12:10,6,0.0,-1.000000,阿華食堂,軟訊資訊有限公司,Family Mart,Graffee Cafe 咖啡街35弄1號,TOYOTA 內湖營業所,restaurant,electronics_store
4,5,121.563,25.087,2020-09-04,12:10,6,0.0,-1.000000,阿華食堂,軟訊資訊有限公司,Family Mart,Graffee Cafe 咖啡街35弄1號,TOYOTA 內湖營業所,restaurant,electronics_store
5,6,121.563,25.087,2020-09-04,12:10,6,0.0,-1.000000,阿華食堂,軟訊資訊有限公司,Family Mart,Graffee Cafe 咖啡街35弄1號,TOYOTA 內湖營業所,restaurant,electronics_store
6,7,121.563,25.087,2020-09-04,12:10,6,0.0,-1.000000,阿華食堂,軟訊資訊有限公司,Family Mart,TOYOTA 內湖營業所,汎美自助餐,restaurant,electronics_store
7,8,121.563,25.087,2020-09-04,12:10,6,0.0,-1.000000,阿華食堂,軟訊資訊有限公司,Family Mart,TOYOTA 內湖營業所,汎美自助餐,restaurant,electronics_store
8,9,121.563,25.087,2020-09-04,12:10,6,0.0,-1.000000,阿華食堂,軟訊資訊有限公司,Family Mart,TOYOTA 內湖營業所,汎美自助餐,restaurant,electronics_store
9,10,121.563,25.087,2020-09-04,12:11,6,0.0,-1.000000,阿華食堂,軟訊資訊有限公司,Family Mart,三顧茅廬-文湖店,義麵小舖,restaurant,electronics_store


In [117]:
for i in range(10):
    dropList = []
    for row in range(1,len(stopLoc)-1): 
        if stopLoc.iloc[row-1,14] == 'entering' and stopLoc.iloc[row,14] != 'exiting':
            dropList.append(row)
            
    print(dropList)
    stopLoc = stopLoc.drop(dropList)
    stopLoc = stopLoc.reset_index(drop = True)
    
stopLoc    

[368, 379, 535, 747, 770, 782, 849, 856, 962, 1056]
[368, 378, 533, 744, 766, 777, 843, 849, 954, 1047]
[377, 531, 741, 762, 772, 837, 842, 946, 1038]
[530, 739, 759, 768, 832, 836, 939, 1030]
[530, 738, 757, 765, 831, 933, 1023]
[530, 737, 755, 827, 928, 1017]
[736, 753, 824, 1012]
[822, 1009]
[822]
[822]


,location_id,longitude,latitude,start_date,start_time,weekday,duration,speed,name1,name2,name3,name4,name5,category1,category2
0,1,121.576,24.986,2020/07/22,11:00,4,NaN,NaN,myhome,myhome1,myhome2,myhome3,myhome4,c1,c2
1,2,121.443,24.433,2020/07/23,12:00,5,NaN,NaN,apple,apple1,apple2,apple3,apple4,c1,c2
2,3,121.123,24.765,2020/07/24,13:00\n,6,NaN,NaN,banana,banana1,banana2,banana3,banana4,c1,c2
3,4,121.563,25.087,2020-09-04,12:10,6,0.0,-1.000000,阿華食堂,軟訊資訊有限公司,Family Mart,Graffee Cafe 咖啡街35弄1號,TOYOTA 內湖營業所,restaurant,electronics_store
4,5,121.563,25.087,2020-09-04,12:10,6,0.0,-1.000000,阿華食堂,軟訊資訊有限公司,Family Mart,Graffee Cafe 咖啡街35弄1號,TOYOTA 內湖營業所,restaurant,electronics_store
5,6,121.563,25.087,2020-09-04,12:10,6,0.0,-1.000000,阿華食堂,軟訊資訊有限公司,Family Mart,Graffee Cafe 咖啡街35弄1號,TOYOTA 內湖營業所,restaurant,electronics_store
6,7,121.563,25.087,2020-09-04,12:10,6,0.0,-1.000000,阿華食堂,軟訊資訊有限公司,Family Mart,TOYOTA 內湖營業所,汎美自助餐,restaurant,electronics_store
7,8,121.563,25.087,2020-09-04,12:10,6,0.0,-1.000000,阿華食堂,軟訊資訊有限公司,Family Mart,TOYOTA 內湖營業所,汎美自助餐,restaurant,electronics_store
8,9,121.563,25.087,2020-09-04,12:10,6,0.0,-1.000000,阿華食堂,軟訊資訊有限公司,Family Mart,TOYOTA 內湖營業所,汎美自助餐,restaurant,electronics_store
9,10,121.563,25.087,2020-09-04,12:11,6,0.0,-1.000000,阿華食堂,軟訊資訊有限公司,Family Mart,三顧茅廬-文湖店,義麵小舖,restaurant,electronics_store


In [118]:
#delete trivial data
for i in range(3):
    dropList = []
    for row in range(1,len(stopLoc)-1): 
        if stopLoc.iloc[row,1] == stopLoc.iloc[row-1,1] and stopLoc.iloc[row,2] == stopLoc.iloc[row-1,2]:
            if stopLoc.iloc[row,14] != 'entering' and stopLoc.iloc[row,14] != 'exiting':
                dropList.append(row)
    
        #compared with the last record, over 2 place name duplicated will be deleted
        res = set(stopLoc.iloc[row,8:13]) & set(stopLoc.iloc[row-1,8:13])
        if len(res) > 1: 
            if stopLoc.iloc[row,14] != 'entering' and stopLoc.iloc[row,14] != 'exiting':
                dropList.append(row)
                
    stopLoc = stopLoc.drop(dropList)
    stopLoc = stopLoc.reset_index(drop = True)
    
stopLoc

,location_id,longitude,latitude,start_date,start_time,weekday,duration,speed,name1,name2,name3,name4,name5,category1,category2
0,1,121.576,24.986,2020/07/22,11:00,4,NaN,NaN,myhome,myhome1,myhome2,myhome3,myhome4,c1,c2
1,2,121.443,24.433,2020/07/23,12:00,5,NaN,NaN,apple,apple1,apple2,apple3,apple4,c1,c2
2,3,121.123,24.765,2020/07/24,13:00\n,6,NaN,NaN,banana,banana1,banana2,banana3,banana4,c1,c2
3,4,121.563,25.087,2020-09-04,12:10,6,0.0,-1.000000,阿華食堂,軟訊資訊有限公司,Family Mart,Graffee Cafe 咖啡街35弄1號,TOYOTA 內湖營業所,restaurant,electronics_store
4,209,121.562,25.087,2020-09-04,18:08,6,0.0,-1.000000,漢豐汽車,LOUISA COFFEE 路易莎咖啡 (內湖文湖門市),全聯福利中心Pxmart 內湖文湖,TOYOTA 內湖營業所,詠大直,car_repair,cafe
5,210,121.563,25.087,2020-09-04,18:09,6,0.0,-1.000000,家,2,NaN,NaN,NaN,home,exiting
6,211,121.561,25.086,2020-09-04,18:09,6,0.0,-1.000000,建安輪胎有限公司,Wen Hu Elementary School,米其林馳加汽車服務中心(建凱店),緹堡汽車服務,全聯福利中心Pxmart 內湖文湖,car_repair,primary_school
7,212,121.560,25.085,2020-09-04,18:09,6,0.0,-1.000000,CPC,尋味食堂關東煮本舖,Wen Hu Elementary School,永祥西點麵包店,萬佳鄉（永群店）,gas_station,restaurant
8,213,121.559,25.085,2020-09-04,18:10,6,0.0,-1.000000,Sym,富麗米雜貨店,內湖家畜醫院,Martial Armour Dazhi,N°168 Prime Steakhouse 牛排館,car_repair,grocery_or_supermarket
9,214,121.558,25.085,2020-09-04,18:10,6,0.0,-1.000000,Grand Victoria Hotel Taipei,N°168 Prime Steakhouse 牛排館,Miramar,la FESTA義大利餐廳,Xi Garden,lodging,restaurant


In [119]:
#calculate duration
import datetime as dt
from datetime import datetime

dropList = []
for row in range(0,len(stopLoc)-1): 
        
    res = set(stopLoc.iloc[row,8:13]) & set(stopLoc.iloc[row-1,8:13])
    if len(res) > 1: 
        if stopLoc.iloc[row,14] != 'entering' and stopLoc.iloc[row,14] != 'exiting':
            dropList.append(row)

    #calculate duration
    if stopLoc.iloc[row+1,3] == stopLoc.iloc[row,3]:
        stopLoc.iloc[row,6] = datetime.strptime(stopLoc.iloc[row+1,4], '%H:%M') - datetime.strptime(stopLoc.iloc[row,4], '%H:%M')       
    else:
        a = dt.datetime(2000,1,2,int(stopLoc.iloc[row+1,4].split(':')[0]),int(stopLoc.iloc[row+1,4].split(':')[1]))
        b = dt.datetime(2000,1,1,int(stopLoc.iloc[row,4].split(':')[0]),int(stopLoc.iloc[row,4].split(':')[1]))
        stopLoc.iloc[row,6] = a - b
            
    #kill duration <10
    if pd.Timedelta(stopLoc.iloc[row,6]) < pd.Timedelta('00:10:00'):
        if stopLoc.iloc[row,14] != 'entering' and stopLoc.iloc[row,14] != 'exiting':
            dropList.append(row)     


removeList = []
for i in range(1,len(dropList)-1):
    if dropList[i] - dropList[i-1] < 3:
        removeList.append(dropList[i])

stopLoc = stopLoc.drop(removeList)
stopLoc = stopLoc.reset_index(drop = True)

for row in range(0,len(stopLoc)-1): 

    #calculate duration
    if stopLoc.iloc[row+1,3] == stopLoc.iloc[row,3]:
        stopLoc.iloc[row,6] = datetime.strptime(stopLoc.iloc[row+1,4], '%H:%M') - datetime.strptime(stopLoc.iloc[row,4], '%H:%M')       
    else:
        a = dt.datetime(2000,1,2,int(stopLoc.iloc[row+1,4].split(':')[0]),int(stopLoc.iloc[row+1,4].split(':')[1]))
        b = dt.datetime(2000,1,1,int(stopLoc.iloc[row,4].split(':')[0]),int(stopLoc.iloc[row,4].split(':')[1]))
        stopLoc.iloc[row,6] = a - b

stopLoc = stopLoc.drop(['start_date','name2','name3','name4','name5','speed'], 1)
stopLoc

,location_id,longitude,latitude,start_time,weekday,duration,name1,category1,category2
0,1,121.576,24.986,11:00,4,"1 day, 1:00:00",myhome,c1,c2
1,2,121.443,24.433,12:00,5,"1 day, 1:00:00",apple,c1,c2
2,3,121.123,24.765,13:00\n,6,23:10:00,banana,c1,c2
3,4,121.563,25.087,12:10,6,5:58:00,阿華食堂,restaurant,electronics_store
4,209,121.562,25.087,18:08,6,0:01:00,漢豐汽車,car_repair,cafe
5,210,121.563,25.087,18:09,6,0:20:00,家,home,exiting
6,250,121.544,25.041,18:29,6,0:14:00,[JJ HAIR SALON 髮型沙龍],beauty_salon,jewelry_store
7,253,121.545,25.025,18:43,6,0:34:00,澳客炙燒牛排,restaurant,meal_takeaway
8,275,121.528,25.025,19:17,6,0:58:00,Shida Triangle Park,park,park
9,276,121.529,25.024,20:15,6,0:23:00,小老婆相機,electronics_store,point_of_interest


In [120]:
for row in range(0,len(stopLoc)-1): 

    #calculate duration
    if abs(stopLoc.iloc[row+1,1] - stopLoc.iloc[row,1]) > 0.03 or abs(stopLoc.iloc[row+1,2] - stopLoc.iloc[row,2]) > 0.03:
        if stopLoc.iloc[row,8] != 'entering':
            stopLoc.iloc[row,7] = 'commute'
stopLoc

,location_id,longitude,latitude,start_time,weekday,duration,name1,category1,category2
0,1,121.576,24.986,11:00,4,"1 day, 1:00:00",myhome,commute,c2
1,2,121.443,24.433,12:00,5,"1 day, 1:00:00",apple,commute,c2
2,3,121.123,24.765,13:00\n,6,23:10:00,banana,commute,c2
3,4,121.563,25.087,12:10,6,5:58:00,阿華食堂,restaurant,electronics_store
4,209,121.562,25.087,18:08,6,0:01:00,漢豐汽車,car_repair,cafe
5,210,121.563,25.087,18:09,6,0:20:00,家,commute,exiting
6,250,121.544,25.041,18:29,6,0:14:00,[JJ HAIR SALON 髮型沙龍],beauty_salon,jewelry_store
7,253,121.545,25.025,18:43,6,0:34:00,澳客炙燒牛排,restaurant,meal_takeaway
8,275,121.528,25.025,19:17,6,0:58:00,Shida Triangle Park,park,park
9,276,121.529,25.024,20:15,6,0:23:00,小老婆相機,electronics_store,point_of_interest


## Map Category

In [96]:
stopLoc = pd.merge(stopLoc, loc_to_cat, how='left', on='category1' )
stopLoc = stopLoc.drop(['name1','category1','location_id'],1)
stopLoc['user_id'] = 2
stopLoc = stopLoc[['user_id','weekday','start_time','duration','location']]
stopLoc
#把start_date加回來
#name1,category1拿掉 place_cat對表換成數字

,user_id,weekday,start_time,duration,location
0,2,4,11:00,"1 day, 1:00:00",NaN
1,2,5,12:00,"1 day, 1:00:00",NaN
2,2,6,13:00\n,23:10:00,NaN
3,2,6,12:10,5:58:00,5.0
4,2,6,18:08,0:01:00,NaN
5,2,6,18:09,0:20:00,1.0
6,2,6,18:29,0:14:00,NaN
7,2,6,18:43,0:34:00,5.0
8,2,6,19:17,0:58:00,19.0
9,2,6,20:15,0:23:00,NaN


In [ ]:
traing = stopLoc.iloc[:, np.r_[0:4]].values
predictions = clf.predict(traing)

## Myplace

In [ ]:
savedplace = pd.read_csv("savedPlace.csv")
savedplace['place_longitude'] = round(saveplace['place_longitude'],3)
savedplace['place_latitude'] = round(saveplace['place_latitude'],3)
#savedplace = savedplace.loc[savedplace['my_place'] == 1]
savedplace = savedplace.drop(['place_name','my_place','place_id'],1)
savedplace = savedplace.rename(columns={"place_longitude": "longitude", "place_latitude": "latitude"})
savedplace = savedplace.drop_duplicates(subset=['longitude', 'latitude'], keep='last')
savedplace.head()

In [ ]:
stopLoc = pd.merge(stopLoc, savedplace, how='left', on=['longitude','latitude'])
stopLoc